In [9]:
import re
import time
import datetime
import csv
import requests

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # для парсера HTML
import openpyxl # для парсера XLSX (Excel)

import pymorphy3 # работа с русским языком, pymorphy3

import texttable as tt

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder # Кодирование категориальных данных

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler # Масштабирование данных

from sklearn.feature_selection import SelectKBest # Выбор признаков с наивысшими оценками
from sklearn.feature_selection import chi2 # Выбор признаков по Хи квадрат

from sklearn.model_selection import train_test_split # Деление выборки на тестовые и тренировочные данные
from sklearn.model_selection import cross_val_score # Оценка качества работы модели

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Критерий качества, точности

from sklearn.neighbors import KNeighborsClassifier # Обучение модели K-ближайших соседей
from sklearn.linear_model import LinearRegression # Линейная регрессия (метод наименьших квадратов)

from sklearn.tree import DecisionTreeClassifier # Деревья решений
from sklearn.ensemble import RandomForestClassifier # Ансамбли деревьев решений
from sklearn.ensemble import GradientBoostingClassifier # Ансамбли градиентного спуска

from sklearn.ensemble import RandomForestRegressor # случайный лес
from sklearn.neighbors import KNeighborsRegressor # метод ближайших соседей
from sklearn.svm import SVR # метод опорных векторов с линейным ядром
from sklearn.linear_model import LogisticRegression # логистическая регрессия

from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif # Статистический метод
from sklearn.metrics import r2_score

# Подготавливаю данные о погоде в "село Сембург", близлежайшее поселение к нефтянной скважины №807 (в 24 км). Разрабатываю парсер сайтов о погоде.

In [16]:
# проведу осмотр данных прогноза погоды в с. Самбург на сайте Яндекс.Погода за текущий год (на данный момент это 2023 год)
# lat=66.80035786146756 (Широта)
# lng=78.38975066524623 (Долгота)

month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

morph = pymorphy3.MorphAnalyzer(lang='ru')

data = []
data_i = 0
for j, item in enumerate(month):
  url = 'https://yandex.ru/pogoda/month/{item}?lat={latitude}&lon={lontitude}'
  url = url.format(item=item, latitude=66.80035786146756, lontitude=78.38975066524623)
  print(url)
  page = requests.get(url)
  bs = BeautifulSoup(page.text, 'html.parser')

  # информация о погоде в с. Самбург за каждый месяц
  for i, val in enumerate(bs.find('table', {'class': 'climate-calendar'}).find_all('td', {'class': 'climate-calendar__cell'})):
    temp = []
    if val.find('div', {'class': 'climate-calendar-day_colorless_yes'}):
      continue
    temp.append(int(data_i))
    temp.append(int(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[0]))
    temp.append(str(morph.parse(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[1])[0].normal_form))
    temp.append(2023)
    temp.append(str(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[1]))
    temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-day'}).find('span').text.replace('−', '-')))
    temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-night'}).find('span').text.replace('−', '-')))
    temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[0].text.split(' ', 1)[0]))
    temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[1].text.split('%', 1)[0]) / 100)
    temp.append(float(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('div').text))
    temp.append(str(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('abbr').text))
    print(temp)
    data.append(temp)
    data_i += 1

# Датафрейм из массива
df = pd.DataFrame(data, columns=['id', 'День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])
df

https://yandex.ru/pogoda/month/january?lat=66.80035786146756&lon=78.38975066524623
[0, 1, 'январь', 2023, ' вс', -21, -26, 760, 0.76, 5.3, 'ЮВ']
[1, 2, 'январь', 2023, ' пн', -22, -22, 763, 0.79, 5.3, 'З']
[2, 3, 'январь', 2023, ' вт', -21, -22, 764, 0.79, 5.8, 'Ю']
[3, 4, 'январь', 2023, ' ср', -17, -20, 762, 0.81, 6.5, 'Ю']
[4, 5, 'январь', 2023, ' чт', -19, -22, 764, 0.8, 6.0, 'Ю']
[5, 6, 'январь', 2023, ' пт', -20, -22, 766, 0.79, 5.5, 'Ю']
[6, 7, 'январь', 2023, ' сб', -19, -21, 766, 0.76, 4.8, 'Ю']
[7, 8, 'январь', 2023, ' вс', -20, -22, 763, 0.76, 5.3, 'ЮВ']
[8, 9, 'январь', 2023, ' пн', -22, -22, 761, 0.77, 5.5, 'Ю']
[9, 10, 'январь', 2023, ' вт', -21, -24, 760, 0.77, 5.8, 'Ю']
[10, 11, 'январь', 2023, ' ср', -24, -24, 762, 0.76, 5.1, 'Ю']
[11, 12, 'январь', 2023, ' чт', -21, -21, 758, 0.77, 6.5, 'Ю']
[12, 13, 'январь', 2023, ' пт', -19, -22, 756, 0.8, 6.4, 'Ю']
[13, 14, 'январь', 2023, ' сб', -20, -20, 757, 0.79, 5.4, 'ЮВ']
[14, 15, 'январь', 2023, ' вс', -17, -20, 759, 0.8, 4

,id,День,Месяц,Год,День недели,Температура днем,Температура ночью,Давление (мм рт. ст.),Влажность (%),Скорость ветра,Направление ветра
0,0,1,январь,2023,вс,-21,-26,760,0.76,5.3,ЮВ
1,1,2,январь,2023,пн,-22,-22,763,0.79,5.3,З
2,2,3,январь,2023,вт,-21,-22,764,0.79,5.8,Ю
3,3,4,январь,2023,ср,-17,-20,762,0.81,6.5,Ю
4,4,5,январь,2023,чт,-19,-22,764,0.80,6.0,Ю
...,...,...,...,...,...,...,...,...,...,...,...
360,360,27,декабрь,2023,ср,-18,-20,753,0.81,5.5,Ю
361,361,28,декабрь,2023,чт,-18,-20,756,0.81,5.0,Ю
362,362,29,декабрь,2023,пт,-19,-21,759,0.81,5.5,Ю
363,363,30,декабрь,2023,сб,-19,-20,760,0.80,5.8,Ю


In [17]:
# cоздаю объект Texttable
tab = tt.Texttable()

# устанавливаю стили таблицы
tab.set_cols_align(['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'])
tab.header(['id', 'День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])

# преобразую DataFrame в список списков (двумерный список)
data_list = df.values.tolist()

# добавляю данные в таблицу
for row in data_list:
  tab.add_row(row)

# получаю отформатированную таблицу в виде строки
table_string = tab.draw()

# вывожу таблицу на экран
print(table_string)

+-----+------+-------+------+-------+-------+------+------+------+------+------+
| id  | День | Месяц | Год  | День  | Темпе | Темп | Давл | Влаж | Скор | Напр |
|     |      |       |      | недел | ратур | ерат | ение | ност | ость | авле |
|     |      |       |      |   и   |   а   | ура  | (мм  |  ь   | ветр | ние  |
|     |      |       |      |       | днем  | ночь | рт.  | (%)  |  а   | ветр |
|     |      |       |      |       |       |  ю   | ст.) |      |      |  а   |
+=====+======+=======+======+=======+=======+======+======+======+======+======+
|  0  |  1   | январ | 2023 |   вс  |  -21  | -26  | 760  | 0.76 | 5.30 |  ЮВ  |
|     |      |   ь   |      |       |       |      |      |  0   |  0   |      |
+-----+------+-------+------+-------+-------+------+------+------+------+------+
|  1  |  2   | январ | 2023 |   пн  |  -22  | -22  | 763  | 0.79 | 5.30 |  З   |
|     |      |   ь   |      |       |       |      |      |  0   |  0   |      |
+-----+------+-------+------

# Как видно я получил данные с Яндекс.Погода за 2023 года, мне нужно больше данных. У Яндекс.Погода нет архивных данных, поэтому буду использовать другой сайт с архивными данными.

In [22]:
# рассмотрю данные прогноза погоды в с. Самбург с сайта Погода1 (так как в нем есть архивные данные)
url = 'https://pogoda1.ru/samburg/arkhiv/'
page = requests.get(url)
bs = BeautifulSoup(page.text, 'html.parser')

# Формирую список диапозона даты, где имеются архивные данные
year = bs.find('select', {'class': 'select-archive-year'}).find_all('option')
month = bs.find('select', {'class': 'select-archive-month'}).find_all('option')

# Перевод месяцев с русского на английский язык
def month_translate(month):
  month = month.lower()
  if month == "январь":
    return "january"
  elif month == "февраль":
    return "february"
  elif month == "март":
    return "march"
  elif month == "апрель":
    return "april"
  elif month == "май":
    return "may"
  elif month == "июнь":
    return "june"
  elif month == "июль":
    return "july"
  elif month == "август":
    return "august"
  elif month == "сентябрь":
    return "september"
  elif month == "октябрь":
    return "october"
  elif month == "ноябрь":
    return "november"
  elif month == "декабрь":
    return "december"

base_month = []
for i, val_year in enumerate(year):
  for j, val_month in enumerate(month):
    temp_0 = []
    if j > 0:
      temp_0.append(int(val_year.text))
      temp_0.append(str(month_translate(month = val_month.text)).lower())
      base_month.append(temp_0)

# Формирую список URL адресов с данными
url_parse = 'https://pogoda1.ru/samburg/{month}-{year}/'

for i, val in enumerate(base_month):
  parse = url_parse.format(month=val[1], year=val[0])
  base_month[i].append(parse)

df = pd.DataFrame(base_month, columns=['Год', 'Месяц', 'URL'])
df


,Год,Месяц,URL
0,2017,january,https://pogoda1.ru/samburg/january-2017/
1,2017,february,https://pogoda1.ru/samburg/february-2017/
2,2017,march,https://pogoda1.ru/samburg/march-2017/
3,2017,april,https://pogoda1.ru/samburg/april-2017/
4,2017,may,https://pogoda1.ru/samburg/may-2017/
...,...,...,...
79,2023,august,https://pogoda1.ru/samburg/august-2023/
80,2023,september,https://pogoda1.ru/samburg/september-2023/
81,2023,october,https://pogoda1.ru/samburg/october-2023/
82,2023,november,https://pogoda1.ru/samburg/november-2023/


In [23]:
# Перевод месяцев с текста на цифры
def month_number(month):
  month = month.lower()
  if month == "january":
    return "01"
  elif month == "february":
    return "02"
  elif month == "march":
    return "03"
  elif month == "april":
    return "04"
  elif month == "may":
    return "05"
  elif month == "june":
    return "06"
  elif month == "july":
    return "07"
  elif month == "august":
    return "08"
  elif month == "september":
    return "09"
  elif month == "october":
    return "10"
  elif month == "november":
    return "11"
  elif month == "december":
    return "12"

# Поиск существующих ссылок с данными
base_day = []
for i, val in enumerate(base_month):
  bs = BeautifulSoup(requests.get(val[2]).text, 'html.parser')
  if (bs.find('div', {'class': 'month-calendar calendar'})):
    print(val[2])
    for j, item in enumerate(bs.find_all('a', {'class': 'calendar-item'})):
      temp = []
      if not (item.find('span', {'class': 'no-data'})):
        temp.append(int(val[0]))
        temp.append(int(month_number(val[1])))
        temp.append(int(item.find('span', {'month-calendar-day'}).text))
        temp.append('https://pogoda1.ru' + str(item['href']))
        print(temp)
        base_day.append(temp)

https://pogoda1.ru/samburg/june-2017/
[2017, 6, 7, 'https://pogoda1.ru/samburg/07-06-2017/']
[2017, 6, 8, 'https://pogoda1.ru/samburg/08-06-2017/']
[2017, 6, 9, 'https://pogoda1.ru/samburg/09-06-2017/']
[2017, 6, 10, 'https://pogoda1.ru/samburg/10-06-2017/']
[2017, 6, 11, 'https://pogoda1.ru/samburg/11-06-2017/']
[2017, 6, 12, 'https://pogoda1.ru/samburg/12-06-2017/']
[2017, 6, 13, 'https://pogoda1.ru/samburg/13-06-2017/']
[2017, 6, 14, 'https://pogoda1.ru/samburg/14-06-2017/']
[2017, 6, 15, 'https://pogoda1.ru/samburg/15-06-2017/']
[2017, 6, 16, 'https://pogoda1.ru/samburg/16-06-2017/']
[2017, 6, 17, 'https://pogoda1.ru/samburg/17-06-2017/']
[2017, 6, 18, 'https://pogoda1.ru/samburg/18-06-2017/']
[2017, 6, 19, 'https://pogoda1.ru/samburg/19-06-2017/']
[2017, 6, 20, 'https://pogoda1.ru/samburg/20-06-2017/']
[2017, 6, 21, 'https://pogoda1.ru/samburg/21-06-2017/']
[2017, 6, 22, 'https://pogoda1.ru/samburg/22-06-2017/']
[2017, 6, 23, 'https://pogoda1.ru/samburg/23-06-2017/']
[2017, 6, 24,

In [104]:
# Парсер погоды
data = []
for i, val in enumerate(base_day):
  # if i < 1593:
  #   continue
  temp = []
  bs = BeautifulSoup(requests.get(val[3]).text, 'html.parser')
  if not (bs.find('div', {'class': 'panel-heading'}).text == '404 Страница не найдена'):
    # print(val[3])
    temp.append(i)
    temp.append(val[0])
    temp.append(val[1])
    temp.append(val[2])
    temp.append(str(bs.find('img', {'class': 'weather-now-icon-img'})['title']).lower())
    if (bs.find('div', {'class': 'weather-now-col weather-now-col-main'}).find('span', {'class': 'wind-amount'})):
      temp.append(str(bs.find_all('span', {'class': 'wind-amount'})[0].text.split(',', 1)[0]).lower())
      temp.append(int(bs.find_all('span', {'class': 'wind-amount'})[0].text.split(',', 1)[1].split(' ', 2)[1]))
    else:
      for j, item in enumerate(bs.find_all('div', {'row-forecast-time-of-day'})):
        if not (str(item.find('div', {'class': 'cell-forecast-wind'}).find('span', {'class': 'wind'}).text) == 'нет'):
          temp.append(str(item.find('div', {'class': 'cell-forecast-wind'}).find('img', {'class': 'icon-wind'})['title'].split(' ', 1)[0]).lower())
          temp.append(int(item.find('div', {'class': 'cell-forecast-wind'}).find('span', {'class': 'wind-amount'}).text.split(' ', 1)[0]))
        else:
          temp.append('')
          temp.append('')
        break
    temp.append(int(bs.find_all('div', {'class': 'weather-now-info'})[0].find('span', {'class': 'value'}).text.split(' ', 1)[0]))
    temp.append(int(bs.find_all('div', {'class': 'weather-now-info'})[1].find('span', {'class': 'value'}).text.split('%', 1)[0]))
    temp.append(float(bs.find_all('div', {'class': 'weather-now-info'})[2].find('span', {'class': 'value'}).text.split(' ', 1)[0]))
    temp.append(str(bs.find_all('div', {'class': 'weather-now-info'})[6].find('span', {'class': 'value'}).text.split(' ', 1)[0]).lower())
    # temp.append(int(bs.find_all('div', {'class': 'weather-now-info'})[6].find('span', {'class': 'value'}).text.split(' ', 1)[1].split('%', 1)[0]))
    if not (str(bs.find_all('div', {'row-forecast-time-of-day'})[2].find('div', {'class': 'cell-forecast-prec opened'}))):
      if (str(bs.find_all('div', {'row-forecast-time-of-day'})[2].find('div', {'class': 'cell-forecast-prec'}).text) == 'без осадков'):
        temp.append(0)
      else:
        temp.append(float(bs.find_all('div', {'row-forecast-time-of-day'})[2].find('div', {'class': 'cell-forecast-prec'}).text.split(' ', 1)[0]))
    else:
      if (str(bs.find_all('div', {'row-forecast-time-of-day'})[1].find('div', {'class': 'cell-forecast-prec'}).text) == 'без осадков'):
        temp.append(0)
      else:
        temp.append(float(bs.find_all('div', {'row-forecast-time-of-day'})[1].find('div', {'class': 'cell-forecast-prec'}).text.split(' ', 1)[0]))
    temp.append(int(bs.find_all('div', {'row-forecast-time-of-day'})[0].find('div', {'class': 'cell-forecast-temp'}).text.split('°', 1)[0]))
    if not (str(bs.find_all('div', {'row-forecast-time-of-day'})[2].find('div', {'class': 'cell-forecast-prec opened'}))):
      temp.append(int(bs.find_all('div', {'row-forecast-time-of-day'})[2].find('div', {'class': 'cell-forecast-temp'}).text.split('°', 1)[0]))
    else:
      temp.append(int(bs.find_all('div', {'row-forecast-time-of-day'})[1].find('div', {'class': 'cell-forecast-temp'}).text.split('°', 1)[0]))
    temp.append(val[3])
    print(temp)
    data.append(temp)

[0, 2017, 6, 7, 'пасмурно', 'северо-западный', 4, 764, 56, 10.0, 'растущая', 0, 9, 13, 'https://pogoda1.ru/samburg/07-06-2017/']
[1, 2017, 6, 8, 'ясно', 'северо-западный', 3, 762, 63, 10.0, 'растущая', 0, 9, 13, 'https://pogoda1.ru/samburg/08-06-2017/']
[2, 2017, 6, 9, 'малооблачно', 'северный', 3, 756, 51, 10.0, 'полнолуние', 0, 12, 15, 'https://pogoda1.ru/samburg/09-06-2017/']
[3, 2017, 6, 10, 'пасмурно', 'северо-западный', 8, 752, 60, 10.0, 'убывающая', 0, 10, 10, 'https://pogoda1.ru/samburg/10-06-2017/']
[4, 2017, 6, 11, 'пасмурно', 'западный', 6, 756, 50, 10.0, 'убывающая', 0, 4, 2, 'https://pogoda1.ru/samburg/11-06-2017/']
[5, 2017, 6, 12, 'пасмурно', 'южный', 6, 758, 70, 10.0, 'убывающая', 0, 5, 12, 'https://pogoda1.ru/samburg/12-06-2017/']
[6, 2017, 6, 13, 'пасмурно', 'юго-восточный', 2, 758, 100, 10.0, 'убывающая', 1.1, 5, 9, 'https://pogoda1.ru/samburg/13-06-2017/']
[7, 2017, 6, 14, 'пасмурно', 'южный', 9, 759, 65, 10.0, 'убывающая', 0, 6, 15, 'https://pogoda1.ru/samburg/14-0

In [188]:
# Оформляю в DataFrame
df = pd.DataFrame(data, columns=[
  'id',
  'год',
  'месяц',
  'день',
  'погодное условие',
  'направление ветра',
  'скорость ветра (м/с)',
  'давление (мм рт. ст.)',
  'влажность (%)',
  'видимость (мм)',
  'луна',
  'осадки (мм)',
  'температура днем',
  'температура ночью',
  'url'
])
# df.drop(columns=["id"], inplace=True)

# Сохранение данных в csv
df.to_csv('data_samburg_weather.csv', sep=',', encoding='utf-8', index=False)
df

,год,месяц,день,погодное условие,направление ветра,скорость ветра (м/с),давление (мм рт. ст.),влажность (%),видимость (мм),луна,осадки (мм),температура днем,температура ночью,url
0,2017,6,7,пасмурно,северо-западный,4,764,56,10.0,растущая,0.0,9,13,https://pogoda1.ru/samburg/07-06-2017/
1,2017,6,8,ясно,северо-западный,3,762,63,10.0,растущая,0.0,9,13,https://pogoda1.ru/samburg/08-06-2017/
2,2017,6,9,малооблачно,северный,3,756,51,10.0,полнолуние,0.0,12,15,https://pogoda1.ru/samburg/09-06-2017/
3,2017,6,10,пасмурно,северо-западный,8,752,60,10.0,убывающая,0.0,10,10,https://pogoda1.ru/samburg/10-06-2017/
4,2017,6,11,пасмурно,западный,6,756,50,10.0,убывающая,0.0,4,2,https://pogoda1.ru/samburg/11-06-2017/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,2023,11,15,снег,юго-западный,10,763,92,10.0,растущая,0.9,-4,-8,https://pogoda1.ru/samburg/15-11-2023/
2346,2023,11,16,снег,юго-западный,5,762,96,10.0,растущая,1.0,-3,-4,https://pogoda1.ru/samburg/16-11-2023/
2347,2023,11,17,снег,северный,8,759,93,10.0,растущая,1.5,-19,-4,https://pogoda1.ru/samburg/17-11-2023/
2348,2023,11,18,пасмурно,северный,7,765,89,10.0,растущая,0.0,-19,-21,https://pogoda1.ru/samburg/18-11-2023/


# Необходимые данные получил в размере 2350 строк. Сохранил данные в файл и загрузил к себе в Git репозиторий. Дальше идет подготовка данных, а именно просмотр пустот и заполнения средними или наиболее встречающимися значениями, просмотр типов данных.

In [3]:
# Загрузка файла из Git моего репозитория в Pandas
# data_samburg_weather = pd.read_csv('https://raw.githubusercontent.com/SotGE/innopolis2023/main/exam/data_samburg_weather.csv', sep=',', index_col=False, quoting=csv.QUOTE_MINIMAL)
data_samburg_weather = pd.read_csv('https://raw.githubusercontent.com/SotGE/innopolis2023/main/exam/data_samburg_weather.csv', sep=',', index_col=False)
data_samburg_weather

,id,год,месяц,день,погодное условие,направление ветра,скорость ветра (м/с),давление (мм рт. ст.),влажность (%),видимость (мм),луна,осадки (мм),температура днем,температура ночью,url
0,0,2017,6,7,пасмурно,северо-западный,4.0,764,56,10.0,растущая,0.0,9,13,https://pogoda1.ru/samburg/07-06-2017/
1,1,2017,6,8,ясно,северо-западный,3.0,762,63,10.0,растущая,0.0,9,13,https://pogoda1.ru/samburg/08-06-2017/
2,2,2017,6,9,малооблачно,северный,3.0,756,51,10.0,полнолуние,0.0,12,15,https://pogoda1.ru/samburg/09-06-2017/
3,3,2017,6,10,пасмурно,северо-западный,8.0,752,60,10.0,убывающая,0.0,10,10,https://pogoda1.ru/samburg/10-06-2017/
4,4,2017,6,11,пасмурно,западный,6.0,756,50,10.0,убывающая,0.0,4,2,https://pogoda1.ru/samburg/11-06-2017/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,2345,2023,11,15,снег,юго-западный,10.0,763,92,10.0,растущая,0.9,-4,-8,https://pogoda1.ru/samburg/15-11-2023/
2346,2346,2023,11,16,снег,юго-западный,5.0,762,96,10.0,растущая,1.0,-3,-4,https://pogoda1.ru/samburg/16-11-2023/
2347,2347,2023,11,17,снег,северный,8.0,759,93,10.0,растущая,1.5,-19,-4,https://pogoda1.ru/samburg/17-11-2023/
2348,2348,2023,11,18,пасмурно,северный,7.0,765,89,10.0,растущая,0.0,-19,-21,https://pogoda1.ru/samburg/18-11-2023/


In [68]:
# Размер данных (количество строк, колонок)
data_samburg_weather.shape

(2350, 15)

In [136]:
# Просмотр типов данных в датасете
print(data_samburg_weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             2350 non-null   int64  
 1   год                    2350 non-null   int64  
 2   месяц                  2350 non-null   int64  
 3   день                   2350 non-null   int64  
 4   погодное условие       2350 non-null   object 
 5   направление ветра      2348 non-null   object 
 6   скорость ветра (м/с)   2348 non-null   float64
 7   давление (мм рт. ст.)  2350 non-null   int64  
 8   влажность (%)          2350 non-null   int64  
 9   видимость (мм)         2350 non-null   float64
 10  луна                   2350 non-null   object 
 11  осадки (мм)            2350 non-null   float64
 12  температура днем       2350 non-null   int64  
 13  температура ночью      2350 non-null   int64  
 14  url                    2350 non-null   object 
dtypes: f

In [137]:
# Проверка данных

# Количество пустых ячеек
data_samburg_weather.isnull().sum()

Unnamed: 0               0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        2
скорость ветра (м/с)     2
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [131]:
# Количество неопределенные значений (неправильно считанные)
data_samburg_weather.isna().sum()

Unnamed: 0               0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        2
скорость ветра (м/с)     2
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [107]:
# Колличество пустых строк
(data_samburg_weather == "").sum()

Unnamed: 0               0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        0
скорость ветра (м/с)     0
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [4]:
# Заполнение данных

# Заполнение пустых значений - наиболее встречающимся классом
# df['направление ветра'] = df['направление ветра'].replace('', str(df['направление ветра'].value_counts().index[0]))
data_samburg_weather['направление ветра'].fillna(str(data_samburg_weather['направление ветра'].value_counts().index[0]), inplace = True)

# Заполнение пустых значений - наиболее распространенного значения
# data_samburg_weather['скорость ветра (м/с)'] = data_samburg_weather['скорость ветра (м/с)'].replace('', float(data_samburg_weather['скорость ветра (м/с)'].value_counts().idxmax()))
data_samburg_weather['скорость ветра (м/с)'].fillna(float(data_samburg_weather['скорость ветра (м/с)'].value_counts().idxmax()), inplace = True)

data_samburg_weather = data_samburg_weather.astype({'скорость ветра (м/с)': 'float64'})

In [12]:
# Просмотр типов данных в датасете
print(data_samburg_weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2350 non-null   int64  
 1   год                    2350 non-null   int64  
 2   месяц                  2350 non-null   int64  
 3   день                   2350 non-null   int64  
 4   погодное условие       2350 non-null   object 
 5   направление ветра      2350 non-null   object 
 6   скорость ветра (м/с)   2350 non-null   float64
 7   давление (мм рт. ст.)  2350 non-null   int64  
 8   влажность (%)          2350 non-null   int64  
 9   видимость (мм)         2350 non-null   float64
 10  луна                   2350 non-null   object 
 11  осадки (мм)            2350 non-null   float64
 12  температура днем       2350 non-null   int64  
 13  температура ночью      2350 non-null   int64  
 14  url                    2350 non-null   object 
dtypes: f

In [13]:
# Повторная проверка данных

# Количество пустых ячеек
data_samburg_weather.isnull().sum()

id                       0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        0
скорость ветра (м/с)     0
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [14]:
# Количество неопределенные значений (неправильно считанные)
data_samburg_weather.isna().sum()

id                       0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        0
скорость ветра (м/с)     0
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [143]:
# Колличество пустых строк
(data_samburg_weather == '').sum()

Unnamed: 0               0
год                      0
месяц                    0
день                     0
погодное условие         0
направление ветра        0
скорость ветра (м/с)     0
давление (мм рт. ст.)    0
влажность (%)            0
видимость (мм)           0
луна                     0
осадки (мм)              0
температура днем         0
температура ночью        0
url                      0
dtype: int64

In [15]:
# Описательная статистика
data_samburg_weather.describe(include='all', percentiles=[0.1, 0.25,0.5, 0.75, 0.9]).T

,count,unique,top,freq,mean,std,min,10%,25%,50%,75%,90%,max
id,2350.0,NaN,NaN,NaN,1174.5,678.530889,0.0,234.9,587.25,1174.5,1761.75,2114.1,2349.0
год,2350.0,NaN,NaN,NaN,2020.155319,1.881358,2017.0,2018.0,2019.0,2020.0,2022.0,2023.0,2023.0
месяц,2350.0,NaN,NaN,NaN,6.659149,3.390316,1.0,2.0,4.0,7.0,10.0,11.0,12.0
день,2350.0,NaN,NaN,NaN,15.701702,8.793305,1.0,4.0,8.0,16.0,23.0,28.0,31.0
погодное условие,2350,6,пасмурно,1549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
направление ветра,2350,8,южный,445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
скорость ветра (м/с),2350.0,NaN,NaN,NaN,5.511489,2.76689,1.0,2.0,4.0,5.0,7.0,9.0,44.0
давление (мм рт. ст.),2350.0,NaN,NaN,NaN,759.973617,9.550216,729.0,748.0,754.0,759.0,765.0,772.0,806.0
влажность (%),2350.0,NaN,NaN,NaN,79.794468,17.196529,0.0,52.0,70.0,85.0,93.0,97.0,100.0
видимость (мм),2350.0,NaN,NaN,NaN,23.398,86.264654,1.0,10.0,10.0,10.0,10.0,10.0,992.0


# На этом парсер погоды закончен, перехожу к подготовке датасета о нефтянной скважины №807 (данные хранятся в формате xlsx, их переведу в формат csv).

In [175]:
# загружаю xlsx документ и выполняю парсер
wookbook = openpyxl.load_workbook('Oil well.xlsx')
worksheet = wookbook["Oil Well"]
data_xlsx = []
data_i = 0
for row in worksheet.iter_rows(min_row=4, min_col=1, max_col=9):
  temp = []
  temp.append(data_i)
  for j, cell in enumerate(row):
    if j == 0:
      temp.append(cell.value.year)
      temp.append(cell.value.month)
      temp.append(cell.value.day)
    else:
      temp.append(cell.value)
  data_xlsx.append(temp)
  data_i += 1

df = pd.DataFrame(data_xlsx, columns=[
  'id',
  'год',
  'месяц',
  'день',
  'объем нефти (м3/сутки)',
  'объем жидкости (м3/сутки)',
  'объем газа (м3/сутки)',
  'объем воды (м3/сутки)',
  'обводненность (%)',
  'рабочее время',
  'динамический уровень (м)',
  'пластовое давление (атм)'
])
# df.drop(columns=["id"], inplace=True)

# Сохранение данных в csv
df.to_csv('data_oil_well_807.csv', sep=',', encoding='utf-8', index=False)
df

,id,год,месяц,день,объем нефти (м3/сутки),объем жидкости (м3/сутки),объем газа (м3/сутки),объем воды (м3/сутки),обводненность (%),рабочее время,динамический уровень (м),пластовое давление (атм)
0,0,2013,1,1,49,70,13055,21,29,24,1819,214
1,1,2013,1,2,49,70,13055,21,29,24,1836,214
2,2,2013,1,3,49,70,13055,21,29,24,1788,214
3,3,2013,1,4,49,70,13055,21,29,24,1789,214
4,4,2013,1,5,44,70,11768,26,36,24,1825,214
...,...,...,...,...,...,...,...,...,...,...,...,...
2934,2934,2021,1,14,6,20,1593,15,70,16,2101,100
2935,2935,2021,1,15,6,20,1593,15,70,16,2113,100
2936,2936,2021,1,16,6,20,1583,14,70,16,2125,100
2937,2937,2021,1,17,6,20,1573,14,70,16,2125,100


# Необходимые данные получил в размере 2939 строк. Сохранил данные в файл и загрузил к себе в Git репозиторий. Дальше идет подготовка данных, а именно просмотр пустот и заполнения средними или наиболее встречающимися значениями, просмотр типов данных.

In [5]:
# Загрузка файла из Git моего репозитория в Pandas
# data_oil_well_807 = pWd.read_csv('https://raw.githubusercontent.com/SotGE/innopolis2023/main/exam/data_oil_well_807.csv', sep=',', index_col=False, quoting=csv.QUOTE_MINIMAL)
data_oil_well_807 = pd.read_csv('https://raw.githubusercontent.com/SotGE/innopolis2023/main/exam/data_oil_well_807.csv', sep=',', index_col=False)
data_oil_well_807

,id,год,месяц,день,объем нефти (м3/сутки),объем жидкости (м3/сутки),объем газа (м3/сутки),объем воды (м3/сутки),обводненность (%),рабочее время,динамический уровень (м),пластовое давление (атм)
0,0,2013,1,1,49,70,13055,21,29,24,1819,214
1,1,2013,1,2,49,70,13055,21,29,24,1836,214
2,2,2013,1,3,49,70,13055,21,29,24,1788,214
3,3,2013,1,4,49,70,13055,21,29,24,1789,214
4,4,2013,1,5,44,70,11768,26,36,24,1825,214
...,...,...,...,...,...,...,...,...,...,...,...,...
2934,2934,2021,1,14,6,20,1593,15,70,16,2101,100
2935,2935,2021,1,15,6,20,1593,15,70,16,2113,100
2936,2936,2021,1,16,6,20,1583,14,70,16,2125,100
2937,2937,2021,1,17,6,20,1573,14,70,16,2125,100


In [4]:
# Размер данных (количество строк, колонок)
data_oil_well_807.shape

(2939, 12)

In [5]:
# Просмотр типов данных в датасете
print(data_oil_well_807.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2939 entries, 0 to 2938
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   id                         2939 non-null   int64
 1   год                        2939 non-null   int64
 2   месяц                      2939 non-null   int64
 3   день                       2939 non-null   int64
 4   объем нефти (м3/сутки)     2939 non-null   int64
 5   объем жидкости (м3/сутки)  2939 non-null   int64
 6   объем газа (м3/сутки)      2939 non-null   int64
 7   объем воды (м3/сутки)      2939 non-null   int64
 8   обводненность (%)          2939 non-null   int64
 9   рабочее время              2939 non-null   int64
 10  динамический уровень (м)   2939 non-null   int64
 11  пластовое давление (атм)   2939 non-null   int64
dtypes: int64(12)
memory usage: 275.7 KB
None


In [6]:
# Проверка данных

# Количество пустых ячеек
data_oil_well_807.isnull().sum()

id                           0
год                          0
месяц                        0
день                         0
объем нефти (м3/сутки)       0
объем жидкости (м3/сутки)    0
объем газа (м3/сутки)        0
объем воды (м3/сутки)        0
обводненность (%)            0
рабочее время                0
динамический уровень (м)     0
пластовое давление (атм)     0
dtype: int64

In [7]:
# Количество неопределенные значений (неправильно считанные)
data_oil_well_807.isna().sum()

id                           0
год                          0
месяц                        0
день                         0
объем нефти (м3/сутки)       0
объем жидкости (м3/сутки)    0
объем газа (м3/сутки)        0
объем воды (м3/сутки)        0
обводненность (%)            0
рабочее время                0
динамический уровень (м)     0
пластовое давление (атм)     0
dtype: int64

In [8]:
# Колличество пустых строк
(data_oil_well_807 == '').sum()

id                           0
год                          0
месяц                        0
день                         0
объем нефти (м3/сутки)       0
объем жидкости (м3/сутки)    0
объем газа (м3/сутки)        0
объем воды (м3/сутки)        0
обводненность (%)            0
рабочее время                0
динамический уровень (м)     0
пластовое давление (атм)     0
dtype: int64

In [9]:
# Описательная статистика
data_oil_well_807.describe(include='all', percentiles=[0.1, 0.25,0.5, 0.75, 0.9]).T

,count,mean,std,min,10%,25%,50%,75%,90%,max
id,2939.0,1469.000000,848.560546,0.0,293.8,734.5,1469.0,2203.5,2644.2,2938.0
год,2939.0,2016.529092,2.311826,2013.0,2013.0,2015.0,2017.0,2019.0,2020.0,2021.0
месяц,2939.0,6.489622,3.466087,1.0,2.0,3.0,7.0,10.0,11.0,12.0
день,2939.0,15.686288,8.794391,1.0,4.0,8.0,16.0,23.0,28.0,31.0
объем нефти (м3/сутки),2939.0,17.624362,9.689026,0.0,8.0,11.0,15.0,22.0,32.0,49.0
объем жидкости (м3/сутки),2939.0,59.464103,18.634101,12.0,32.0,50.0,58.0,74.0,83.0,113.0
объем газа (м3/сутки),2939.0,4730.146308,2598.888524,4.0,2134.8,3041.5,3909.0,5843.5,8511.0,13113.0
объем воды (м3/сутки),2939.0,41.828853,13.056625,9.0,23.0,33.0,43.0,50.0,58.0,99.0
обводненность (%),2939.0,70.694794,9.534203,29.0,59.0,69.0,73.0,76.0,78.0,100.0
рабочее время,2939.0,22.344675,3.039553,7.0,17.0,22.0,24.0,24.0,24.0,24.0


# На этом парсер нефтянной скважины №807 закончен. Теперь есть 2 датасета: data_samburg_weather.csv и data_oil_well_807.csv

In [6]:
# найду общие даты между ними и обрежу датасеты (объеденение 2х датасетов в один)
# data_samburg_weather
# data_oil_well_807

data = data_samburg_weather.merge(data_oil_well_807, how='inner', on=['год', 'месяц', 'день'])
data


,id_x,год,месяц,день,погодное условие,направление ветра,скорость ветра (м/с),давление (мм рт. ст.),влажность (%),видимость (мм),...,url,id_y,объем нефти (м3/сутки),объем жидкости (м3/сутки),объем газа (м3/сутки),объем воды (м3/сутки),обводненность (%),рабочее время,динамический уровень (м),пластовое давление (атм)
0,0,2017,6,7,пасмурно,северо-западный,4.0,764,56,10.0,...,https://pogoda1.ru/samburg/07-06-2017/,1617,16,52,4210,36,69,24,1841,151
1,1,2017,6,8,ясно,северо-западный,3.0,762,63,10.0,...,https://pogoda1.ru/samburg/08-06-2017/,1618,16,52,4210,36,69,24,1851,151
2,2,2017,6,9,малооблачно,северный,3.0,756,51,10.0,...,https://pogoda1.ru/samburg/09-06-2017/,1619,16,52,4210,36,69,24,1851,151
3,3,2017,6,10,пасмурно,северо-западный,8.0,752,60,10.0,...,https://pogoda1.ru/samburg/10-06-2017/,1620,16,52,4210,36,69,24,1851,151
4,4,2017,6,11,пасмурно,западный,6.0,756,50,10.0,...,https://pogoda1.ru/samburg/11-06-2017/,1621,16,52,4210,36,69,24,1860,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,1310,2021,1,14,снег,западный,8.0,763,95,380.0,...,https://pogoda1.ru/samburg/14-01-2021/,2934,6,20,1593,15,70,16,2101,100
1311,1311,2021,1,15,пасмурно,юго-западный,3.0,770,87,10.0,...,https://pogoda1.ru/samburg/15-01-2021/,2935,6,20,1593,15,70,16,2113,100
1312,1312,2021,1,16,пасмурно,юго-восточный,5.0,759,93,7.4,...,https://pogoda1.ru/samburg/16-01-2021/,2936,6,20,1583,14,70,16,2125,100
1313,1313,2021,1,17,снег,восточный,8.0,747,93,132.0,...,https://pogoda1.ru/samburg/17-01-2021/,2937,6,20,1573,14,70,16,2125,100


In [7]:
# удалю ненужные столбцы
data.drop(columns=[
  'id_x',
  'url',
  'id_y'
], inplace=True)
data

,год,месяц,день,погодное условие,направление ветра,скорость ветра (м/с),давление (мм рт. ст.),влажность (%),видимость (мм),луна,...,температура днем,температура ночью,объем нефти (м3/сутки),объем жидкости (м3/сутки),объем газа (м3/сутки),объем воды (м3/сутки),обводненность (%),рабочее время,динамический уровень (м),пластовое давление (атм)
0,2017,6,7,пасмурно,северо-западный,4.0,764,56,10.0,растущая,...,9,13,16,52,4210,36,69,24,1841,151
1,2017,6,8,ясно,северо-западный,3.0,762,63,10.0,растущая,...,9,13,16,52,4210,36,69,24,1851,151
2,2017,6,9,малооблачно,северный,3.0,756,51,10.0,полнолуние,...,12,15,16,52,4210,36,69,24,1851,151
3,2017,6,10,пасмурно,северо-западный,8.0,752,60,10.0,убывающая,...,10,10,16,52,4210,36,69,24,1851,151
4,2017,6,11,пасмурно,западный,6.0,756,50,10.0,убывающая,...,4,2,16,52,4210,36,69,24,1860,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,2021,1,14,снег,западный,8.0,763,95,380.0,растущая,...,-13,-13,6,20,1593,15,70,16,2101,100
1311,2021,1,15,пасмурно,юго-западный,3.0,770,87,10.0,растущая,...,-20,-21,6,20,1593,15,70,16,2113,100
1312,2021,1,16,пасмурно,юго-восточный,5.0,759,93,7.4,растущая,...,-17,-16,6,20,1583,14,70,16,2125,100
1313,2021,1,17,снег,восточный,8.0,747,93,132.0,растущая,...,-19,-19,6,20,1573,14,70,16,2125,100


# Данные для исседования подготовил.

In [8]:
# Проверка значений на 0
(data == 0).sum()

# Как видно, представленные параметры могут находится в значении 0

# Заполнение нулевых значений - медианой не требуется
# data = data.replace(0, data.median())

год                             0
месяц                           0
день                            0
погодное условие                0
направление ветра               0
скорость ветра (м/с)            0
давление (мм рт. ст.)           0
влажность (%)                   5
видимость (мм)                  0
луна                            0
осадки (мм)                  1033
температура днем               83
температура ночью              77
объем нефти (м3/сутки)          1
объем жидкости (м3/сутки)       0
объем газа (м3/сутки)           0
объем воды (м3/сутки)           0
обводненность (%)               0
рабочее время                   0
динамический уровень (м)        0
пластовое давление (атм)        0
dtype: int64

In [12]:
# Просмотр типов данных в датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   год                        1315 non-null   int64  
 1   месяц                      1315 non-null   int64  
 2   день                       1315 non-null   int64  
 3   погодное условие           1315 non-null   object 
 4   направление ветра          1315 non-null   object 
 5   скорость ветра (м/с)       1315 non-null   float64
 6   давление (мм рт. ст.)      1315 non-null   int64  
 7   влажность (%)              1315 non-null   int64  
 8   видимость (мм)             1315 non-null   float64
 9   луна                       1315 non-null   object 
 10  осадки (мм)                1315 non-null   float64
 11  температура днем           1315 non-null   int64  
 12  температура ночью          1315 non-null   int64  
 13  объем нефти (м3/сутки)     1315 non-null   int64

In [14]:
# Разделение для задачи классификации на X (экзогенные переменные, т.е. регрессоры или независимые) и y (эндогенные переменные или независимые)

# Числовые данные
X = data.drop(columns=['погодное условие', 'направление ветра', 'луна'])

# Категориальные данные
y_1 = data['погодное условие'].astype("category")
y_2 = data['направление ветра'].astype("category")
y_3 = data['луна'].astype("category")

In [15]:
# Построить распределение для всех числовых переменных
figure = px.box(X)
figure.show()

In [17]:
# Подготовка данных

# Нормализация (StandardScaler)
scalar = StandardScaler()
features = scalar.fit_transform(X, y_1)
X_normalised = pd.DataFrame(features, columns=X.columns)
X_normalised

,год,месяц,день,скорость ветра (м/с),давление (мм рт. ст.),влажность (%),видимость (мм),осадки (мм),температура днем,температура ночью,объем нефти (м3/сутки),объем жидкости (м3/сутки),объем газа (м3/сутки),объем воды (м3/сутки),обводненность (%),рабочее время,динамический уровень (м),пластовое давление (атм)
0,-1.607659,-0.248315,-0.988049,-0.457701,0.290160,-1.634375,-0.073229,-0.264322,0.974504,1.051695,1.559505,0.526925,1.473433,0.195407,-1.816550,0.777298,-1.327359,1.726617
1,-1.607659,-0.248315,-0.874296,-0.814264,0.096769,-1.173568,-0.073229,-0.264322,0.974504,1.051695,1.559505,0.526925,1.473433,0.195407,-1.816550,0.777298,-1.246130,1.726617
2,-1.607659,-0.248315,-0.760543,-0.814264,-0.483405,-1.963523,-0.073229,-0.264322,1.196814,1.189298,1.559505,0.526925,1.473433,0.195407,-1.816550,0.777298,-1.246130,1.726617
3,-1.607659,-0.248315,-0.646790,0.968548,-0.870187,-1.371057,-0.073229,-0.264322,1.048607,0.845291,1.559505,0.526925,1.473433,0.195407,-1.816550,0.777298,-1.246130,1.726617
4,-1.607659,-0.248315,-0.533038,0.255423,-0.483405,-2.029352,-0.073229,-0.264322,0.603986,0.294879,1.559505,0.526925,1.473433,0.195407,-1.816550,0.777298,-1.173024,1.726617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,2.127449,-1.700866,-0.191779,0.968548,0.193465,0.932977,7.697880,-0.264322,-0.655774,-0.737144,-1.507723,-1.814468,-1.536277,-1.740090,-1.501409,-1.338948,0.784600,-1.731309
1311,2.127449,-1.700866,-0.078027,-0.814264,0.870334,0.406341,-0.073229,-0.264322,-1.174499,-1.287556,-1.507723,-1.814468,-1.536277,-1.740090,-1.501409,-1.338948,0.882075,-1.731309
1312,2.127449,-1.700866,0.035726,-0.101139,-0.193318,0.801318,-0.127837,-0.264322,-0.952188,-0.943549,-1.507723,-1.814468,-1.547777,-1.832256,-1.501409,-1.338948,0.979550,-1.731309
1313,2.127449,-1.700866,0.149479,0.968548,-1.353665,0.801318,2.489137,-0.264322,-1.100395,-1.149953,-1.507723,-1.814468,-1.559278,-1.832256,-1.501409,-1.338948,0.979550,-1.731309


In [18]:
# Построить распределение для всех числовых нормализированных переменных
figure = px.box(X_normalised)
figure.show()